# 02 - Clean and Normalize

## Overview
Standardize compounds, derive stints, attach tyre age, remove outliers.

## Inputs
- data/raw/*_laps.parquet

## Outputs
- data/interim/laps_interim.parquet
- data/interim/stints_interim.parquet

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat, clean, validation

config.ensure_dirs()

## Load Raw Laps

In [ ]:
raw_dir = config.paths()['data_raw']
laps_files = list(raw_dir.glob('*_laps.parquet'))

print(f"Found {len(laps_files)} lap files")

all_laps = []
for laps_file in laps_files:
    laps = pd.read_parquet(laps_file)
    all_laps.append(laps)
    print(f"  Loaded {laps_file.name}: {len(laps):,} laps")

laps_raw = pd.concat(all_laps, ignore_index=True)
print(f"\nTotal laps: {len(laps_raw):,}")

## Transform: Clean Pipeline

In [ ]:
laps_clean, stints = clean.clean_pipeline(laps_raw)

print(f"\nCleaned laps: {len(laps_clean):,}")
print(f"Stints derived: {len(stints):,}")

## Validate

In [ ]:
# Validate schema
required_lap_cols = ['session_key', 'driver', 'lap', 'compound', 'stint_id', 'tyre_age_laps']
validation.validate_schema(laps_clean, required_lap_cols, name='laps_interim')

# Validate no NAs in key columns
validation.assert_no_na(laps_clean, ['session_key', 'driver', 'lap', 'compound'], name='laps_interim')

# Check compounds
validation.validate_categorical(laps_clean, 'compound', set(config.VALID_COMPOUNDS), name='laps_interim')

print('\n✓ All validations passed')

## Save

In [ ]:
interim_dir = config.paths()['data_interim']

io_flat.write_parquet(laps_clean, interim_dir / 'laps_interim.parquet')
io_flat.write_parquet(stints, interim_dir / 'stints_interim.parquet')

print('\n✓ Saved interim data')

## Repro Notes

- Standardized compounds
- Derived stints based on pit stops and compound changes
- Attached tyre age
- Removed outliers using MAD
- All validations passed